In [42]:
import gym
import ptan
import argparse
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [43]:
#set parameters
num_gps = 3
num_slots = 4
num_pre_booked = 2
num_to_book = 3

In [44]:
GAMMA = 0.99
LEARNING_RATE = 0.001
ENTROPY_BETA = 0.01
BATCH_SIZE = 128
NUM_ENVS = 50

REWARD_STEPS = 4
CLIP_GRAD = 0.1

In [45]:
class PGN(nn.Module):
    def __init__(self, input_size, n_actions):
        super(PGN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.net(x)
        return self.tanh(x)

In [46]:
class SchedulerEnv(gym.Env):
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self):
        
        #set parameters for the day
        self.num_gps = num_gps
        self.num_slots = num_slots
        self.num_pre_booked = num_pre_booked
        self.num_to_book = num_to_book
        self.diary_slots = num_gps*num_slots

        #set action space this format of the diary
        self.action_space = gym.spaces.Box(low=0, high=1, shape=(num_slots, num_gps), dtype=np.int32)
        
        #set observation space 
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(num_slots, num_gps), dtype=np.int32)
   
    #creates daily diary for each gp and randomly populates prebooked appointments
    def reset(self):

        #creates zero filled dataframe with row per time slot and column per gp
        self.state = np.zeros((self.num_slots, self.num_gps),dtype=float)

        #randomly enters a 1 for each pre booked appointments
        while self.num_pre_booked>=0:
            self.num_pre_booked -= 1
            row_to_update = np.random.randint(self.num_slots, size=1)
            col_to_update = np.random.randint(self.num_gps, size=1)
            #self.state.at[row_to_update[0],col_to_update[0]]=1
            self.state[row_to_update,col_to_update]=1

        #resets parameters for new episode
        self.done = False
        self.reward = 0
        self.num_to_book = num_to_book
        self.num_pre_booked = num_pre_booked

        return self.state

    def step(self, action):
    
        tot_appts = self.num_pre_booked + self.num_to_book
        print('total appts to book = ', tot_appts)
        final_diary = action
        print('final appts in diary = ', action.sum())
        
        #rewards if keeps original appointments in same place
        pre_booked_position = np.transpose(np.nonzero(self.state))
        for i in (pre_booked_position):
            if action[i[0],i[1]]:
                self.reward +=1
                print('plus 1 matching')
            else:
                self.reward -=1
                print('minus 1 appt lost')

        #rewards if all new appts are booked
        if tot_appts == action.sum():
            self.reward +=1
            print('plus 1 all booked')
        else:
            self.reward -=1
            print('not all booked')
        
        #rewards if all longer appts are booked together

        self.done = True
        info = {}

        return action, self.reward, self.done, info
        

In [47]:
env = SchedulerEnv()
state = env.reset()
print(state)
device = torch.device("cpu")
net = PGN(env.state.size, env.diary_slots)

[[0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [1. 0. 1.]]


In [48]:
#convert numpy array to tensor for input
def tensor_convert(x):
    return torch.from_numpy(x).float()

In [49]:
nn_input = torch.flatten(tensor_convert(state))
print(nn_input.size())

action = net(nn_input)
dim_action = action.reshape([4,3])
dim_action[dim_action>0] = 1 
dim_action[dim_action<0] = 0 
print(dim_action)

torch.Size([12])
tensor([[0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 1.],
        [1., 1., 0.]], grad_fn=<AsStridedBackward>)


In [50]:
new_state, rewards, done, info = env.step(dim_action)
rewards

total appts to book =  5
final appts in diary =  tensor(6., grad_fn=<SumBackward0>)
plus 1 matching
plus 1 matching
minus 1 appt lost
not all booked


0

In [51]:
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, eps=1e-3)

In [52]:
# train function
def train(memory, q_val):
    values = torch.stack(memory.values)
    q_vals = np.zeros((len(memory), 1))
    
    # target values are calculated backward
    # it's super important to handle correctly done states,
    # for those cases we want our to target to be equal to the reward only
    for i, (_, _, reward, done) in enumerate(memory.reversed()):
        q_val = reward + gamma*q_val*(1.0-done)
        q_vals[len(memory)-1 - i] = q_val # store values from the end to the beginning
        
    advantage = torch.Tensor(q_vals) - values
    
    critic_loss = advantage.pow(2).mean()
    adam_critic.zero_grad()
    critic_loss.backward()
    adam_critic.step()
    
    actor_loss = (-torch.stack(memory.log_probs)*advantage.detach()).mean()
    adam_actor.zero_grad()
    actor_loss.backward()
    adam_actor.step()
